In [ ]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.disable_max_rows()
from datetime import datetime
from scipy.stats import ttest_ind

# A/B test analysis

## Assignments

In [ ]:
data = pd.read_csv("Assignments.csv")

In [ ]:
data.head()

,userid,ts,groupid
0,c5d77c89-33a3-4fe3-9e31-179dec09d49c,2021-11-02T07:31:42Z,0
1,9061d751-7a94-44d3-8792-5ca5ec59aa89,2021-11-13T07:43:51Z,0
2,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-11-20T19:26:07Z,0
3,d2646662-269f-49de-aab1-8776afced9a3,2021-11-20T11:09:02Z,0
4,2d9b23b7-4e5e-4162-9f0f-49e593fdd2b5,2021-11-04T07:42:07Z,0


In [ ]:
 print(datetime.strptime(data.head(1)['ts'][0], '%Y-%m-%dT%H:%M:%SZ').strftime("%Y-%m-%d"))

2021-11-02


In [ ]:
data['dt'] = data['ts'].map(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ').strftime("%Y-%m-%d"))

In [ ]:
data.head()

,userid,ts,groupid,dt
0,c5d77c89-33a3-4fe3-9e31-179dec09d49c,2021-11-02T07:31:42Z,0,2021-11-02
1,9061d751-7a94-44d3-8792-5ca5ec59aa89,2021-11-13T07:43:51Z,0,2021-11-13
2,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-11-20T19:26:07Z,0,2021-11-20
3,d2646662-269f-49de-aab1-8776afced9a3,2021-11-20T11:09:02Z,0,2021-11-20
4,2d9b23b7-4e5e-4162-9f0f-49e593fdd2b5,2021-11-04T07:42:07Z,0,2021-11-04


In [ ]:
data.describe()

,groupid
count,60000.000000
mean,0.500817
std,0.500003
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
data.groupby(['groupid'])["userid"].count()

,userid
groupid,
0,29951
1,30049


In [ ]:
data_count = data.groupby(['groupid','dt'])["userid"].count().reset_index()

In [ ]:
data_count.head()

,groupid,dt,userid
0,0,2021-11-01,1497
1,0,2021-11-02,1467
2,0,2021-11-03,1532
3,0,2021-11-04,1509
4,0,2021-11-05,1503


In [ ]:
alt.Chart(data_count).mark_line(size=3).encode(
    alt.X('dt'),
    alt.Y('userid'),
    color='groupid:O',
    tooltip=['userid']
).properties(
    width=600,
    height=400
)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

## Pre-test metrics

### User activity

In [ ]:
data_act = pd.read_csv("Activity_all.csv")

In [1]:
data_act.head()

NameError: name 'data_act' is not defined

In [ ]:
data_act.groupby(['groupid','dt']).describe()

activity_level                                           \
                            count      mean       std  min  25%  50%   75%   
groupid dt                                                                   
0.0     2021-10-01        27973.0  4.281915  5.610654  0.0  0.0  0.0   9.0   
        2021-10-02        27982.0  4.302838  5.612831  0.0  0.0  0.0   9.0   
        2021-10-03        27978.0  4.310994  5.612682  0.0  0.0  0.0   9.0   
        2021-10-04        28021.0  4.275044  5.625580  0.0  0.0  0.0   9.0   
        2021-10-05        27997.0  4.225596  5.607504  0.0  0.0  0.0   8.0   
...                           ...       ...       ...  ...  ...  ...   ...   
1.0     2021-11-26        24826.0  8.293201  4.764846  0.0  4.0  8.0  12.0   
        2021-11-27        24796.0  8.276133  4.760733  0.0  4.0  8.0  12.0   
        2021-11-28        24837.0  8.230181  4.769722  0.0  4.0  8.0  12.0   
        2021-11-29        24823.0  8.225194  4.793206  0.0  4.0  8.0  12.0   
        2021-11-30        24845.0  8.220326  4.734506  0.0  4.0  8.0  12.0   

                          
                     max  
groupid dt                
0.0     2021-10-01  17.0  
        2021-10-02  17.0  
        2021-10-03  17.0  
        2021-10-04  17.0  
        2021-10-05  17.0  
...                  ...  
1.0     2021-11-26  16.0  
        2021-11-27  16.0  
        2021-11-28  16.0  
        2021-11-29  16.0  
        2021-11-30  16.0  

[122 rows x 8 columns]

In [ ]:
data_act.query('activity_level > 0').groupby(['dt', 'groupid']).count().reset_index().head()

,dt,groupid,userid,activity_level
0,2021-10-01,0.0,13359,13359
1,2021-10-01,1.0,13337,13337
2,2021-10-02,0.0,13385,13385
3,2021-10-02,1.0,13410,13410
4,2021-10-03,0.0,13450,13450


In [ ]:
alt.Chart(data_act.query('activity_level > 0').groupby(['dt', 'groupid']).count().reset_index()).mark_line(size=3).encode(
    alt.X('dt'),
    alt.Y('userid'),
    color='groupid:O',
    tooltip=['userid']
).properties(
    width=900,
    height=600
)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [ ]:
(
    data_act.query('activity_level > 0 and groupid == 0 and dt >= "2021-11-01"')
    .groupby(['dt','groupid']).count().reset_index()[['groupid','activity_level']].describe()
)

,groupid,activity_level
count,30.0,30.000000
mean,0.0,13593.133333
std,0.0,606.542419
min,0.0,12451.000000
25%,0.0,13247.750000
50%,0.0,13944.000000
75%,0.0,14000.000000
max,0.0,14139.000000


In [ ]:
(
    data_act.query('activity_level > 0 and groupid == 1 and dt >= "2021-11-01"')
    .groupby(['dt','groupid']).count().reset_index()[['groupid','activity_level']].describe()
)

,groupid,activity_level
count,30.0,30.000000
mean,1.0,25226.833333
std,0.0,586.172511
min,1.0,24061.000000
25%,1.0,25361.500000
50%,1.0,25515.000000
75%,1.0,25560.750000
max,1.0,25631.000000


In [ ]:
data_act.query('dt >= "2021-11-01"').groupby(['groupid']).describe()

activity_level                                               
                 count      mean       std  min  25%  50%   75%   max
groupid                                                              
0.0           832864.0  4.356597  5.598652  0.0  0.0  0.0   9.0  17.0
1.0           779202.0  8.637338  5.003309  0.0  4.0  9.0  13.0  17.0

In [ ]:
data_act.query('dt < "2021-11-01"').groupby('groupid').describe()

activity_level                                              
                 count      mean       std  min  25%  50%  75%   max
groupid                                                             
0.0           867217.0  4.288029  5.620745  0.0  0.0  0.0  9.0  17.0
1.0           870380.0  4.288553  5.625689  0.0  0.0  0.0  9.0  17.0

In [ ]:
data_act_count = data_act.query('activity_level > 0').groupby(['groupid','dt']).count().reset_index()

In [ ]:
data_act_count.head()

,groupid,dt,userid,activity_level
0,0,2021-10-01,15337,15337
1,0,2021-10-02,15354,15354
2,0,2021-10-03,15423,15423
3,0,2021-10-04,15211,15211
4,0,2021-10-05,15126,15126


In [ ]:
alt.Chart(data_act_count).mark_line(size=3).encode(
    alt.X('dt'),
    alt.Y('userid'),
    color='groupid:O',
    tooltip=['userid']
).properties(
    width=900,
    height=600
)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

### Comparing the activity between the groups

By the activity levels

In [ ]:
data_act.query('groupid == 0')['activity_level'].to_numpy()

array([ 0.,  0.,  0., ..., 17., 17., 17.])

In [ ]:
res = ttest_ind(data_act.query('groupid == 0 and dt >= "2021-11-01"')['activity_level'].to_numpy(),
                data_act.query('groupid == 1 and dt >= "2021-11-01"')['activity_level'].to_numpy()).pvalue

print(res)

0.0


In [ ]:
"{:.100f}".format(res)

'0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

By the number of active users

In [ ]:
before = data_act_count.query('dt < "2021-11-01"')

In [ ]:
after = data_act_count.query('dt >= "2021-11-01"')

In [ ]:
before.head()

,groupid,dt,userid,activity_level
0,0.0,2021-10-01,13359,13359
1,0.0,2021-10-02,13385,13385
2,0.0,2021-10-03,13450,13450
3,0.0,2021-10-04,13281,13281
4,0.0,2021-10-05,13172,13172


Checking for the pretest bias on activity.

In [ ]:
np.mean(before.query('groupid == 0')['userid'].to_numpy())

13344.612903225807

In [ ]:
np.mean(before.query('groupid == 1')['userid'].to_numpy())

13380.290322580646

In [ ]:
res = ttest_ind(before.query('groupid == 0')['userid'].to_numpy(), before.query('groupid == 1')['userid']
                .to_numpy()).pvalue

print(res)

0.09610433277674836


In [ ]:
"{:.100f}".format(res)

'0.0961043327767483607226850494953396264463663101196289062500000000000000000000000000000000000000000000'

In [ ]:
np.mean(after.query('groupid == 0')['userid'].to_numpy())

13593.133333333333

In [ ]:
np.mean(after.query('groupid == 1')['userid'].to_numpy())

25226.833333333332

In [ ]:
res = ttest_ind(after.query('groupid == 0')['userid'].to_numpy(), after.query('groupid == 1')['userid']
                .to_numpy()).pvalue

print(res)

1.2511441823835226e-59


In [ ]:
"{:.100f}".format(res)

'0.0000000000000000000000000000000000000000000000000000000000125114418238352263521567525907719643458854'

### Click through rate (CTR)

In [ ]:
data_ctr = pd.read_csv("Ctr_all.csv")

In [ ]:
data_ctr.head()

,userid,dt,groupid,ctr
0,60389fa7-2d71-4cdf-831c-c2bb277ffa1e,2021-11-13,0,31.81
1,b59cb225-d160-4851-92d2-7cc8120a2f63,2021-11-13,0,30.46
2,aa336050-934e-453f-a5b0-dd881fcd114e,2021-11-13,0,34.25
3,8df767f4-a10f-4322-a722-676b7e02b372,2021-11-13,0,34.92
4,a74762ed-4da0-42ab-91d2-40d7e808dfe9,2021-11-13,0,34.95


In [ ]:
data_ctr_avg = data_ctr.groupby(['groupid','dt'])["ctr"].mean().reset_index()
data_ctr_avg

,groupid,dt,ctr
0,0,2021-10-01,32.980627
1,0,2021-10-02,33.004056
2,0,2021-10-03,33.002006
3,0,2021-10-04,32.990363
4,0,2021-10-05,33.014167
...,...,...,...
117,1,2021-11-26,37.997834
118,1,2021-11-27,37.978912
119,1,2021-11-28,37.992709
120,1,2021-11-29,37.987909


In [ ]:
alt.Chart(data_ctr_avg).mark_line(size=5).encode(
    alt.X('dt'),
    alt.Y('ctr'),
    color='groupid:O',
    tooltip=['ctr']
).properties(
    width=600,
    height=400
)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [ ]:
before = data_ctr.query('dt < "2021-11-01"')[['groupid', 'ctr']]

In [ ]:
after = data_ctr.query('dt >= "2021-11-01"')[['groupid', 'ctr']]

In [ ]:
after

,groupid,ctr
0,0,31.81
1,0,30.46
2,0,34.25
3,0,34.92
4,0,34.95
...,...,...
2303403,1,37.27
2303404,1,39.14
2303405,1,40.05
2303406,1,38.14


In [ ]:
before.query('groupid == 0')['ctr'].to_numpy().mean()

33.00091277553074

In [ ]:
before.query('groupid == 1')['ctr'].to_numpy().mean()

32.99957172093258

In [ ]:
after.query('groupid == 0')['ctr'].to_numpy().mean()

32.996977569382835

In [ ]:
after.query('groupid == 1')['ctr'].to_numpy().mean()

37.99695912626142

In [ ]:
before.query('groupid == 0')['ctr'].to_numpy().std()

1.7336979501682888

In [ ]:
before.query('groupid == 1')['ctr'].to_numpy().std()

1.7296548367391134

In [ ]:
after.query('groupid == 0')['ctr'].to_numpy().std()

1.7331985918552912

In [ ]:
after.query('groupid == 1')['ctr'].to_numpy().std()

1.7323710606903675

In [ ]:
res = ttest_ind(before.query('groupid == 0')['ctr'].to_numpy(), before.query('groupid == 1')['ctr']
                .to_numpy()).pvalue

print(res)

0.705741417344299


In [ ]:
res = ttest_ind(after.query('groupid == 0')['ctr'].to_numpy(), after.query('groupid == 1')['ctr']
                .to_numpy()).pvalue
print(res)

0.0


In [ ]:
"{:.100f}".format(res)

'0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'